In [ ]:
import panel as pn
pn.extension()

import requests

In [ ]:
def get_citing_pmids(doi):
    
    base_url_works = 'https://api.openalex.org/works'
    
    # get work id
    params = {'filter': f'doi:{doi}'}
    r = requests.get(base_url_works, params)
    data = r.json()
    work_id = data['results'][0]['id']  # if multiple, take first
    work_id = work_id.replace('https://openalex.org/', '')
    
    # obtain citing documents/pmids
    params = {'filter': f'cites:{work_id}',
              'cursor': '*', 'per-page': 100}
    pmids = set()
    done = False
    while not done:
        r = requests.get(base_url_works, params)
        data = r.json()
        for work in data['results']:
            pmid = work['ids'].get('pmid')
            if pmid:
                pmids.add(pmid.replace('https://pubmed.ncbi.nlm.nih.gov/', ''))
        if data['meta']['next_cursor']:
            params['cursor'] = data['meta']['next_cursor']
        else:
            done = True
    
    return list(pmids)

In [ ]:
def get_clinical_trials(pmids):
    
    search_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    
    query = ' OR '.join([f'{pmid}[pmid]' for pmid in pmids]) + ' AND (clinicaltrial[Filter])'
                                                                     
    data = {'term': query.encode('utf-8'), 'db': 'pubmed', 'retmax': 10000, 'retmode': 'json'}
    # https://stackoverflow.com/questions/55887958/what-is-the-default-encoding-when-python-requests-post-data-is-string-type
    headers={'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
             'Accept': 'application/json'}
    r = requests.post(search_url, data=data, headers=headers)
    data = r.json()['esearchresult']
    
    return data['idlist']

In [ ]:
def get_metric(doi):
    pmids = get_citing_pmids(doi)
    pmids_t =  get_clinical_trials(pmids)
    return round(len(pmids_t)/len(pmids)*100,2)  

In [ ]:
# Test
#doi = '10.1136/annrheumdis-2019-216655'  # this article is cited by at least 1 clinical trial
#get_metric(doi)

In [ ]:
output_status = pn.pane.Str('')
output_ratio = pn.indicators.Number(name='Ratio', value=0, format='{value}%')
input_doi = pn.widgets.TextInput(placeholder='Enter DOI here...')

def callback(target, event):
    target.object = 'Search in progress...'
    ratio = get_metric(event.new.strip())
    output_ratio.value = ratio
    target.object = 'Done'
    
input_doi.link(output_status, callbacks={'value': callback});

In [ ]:
template = pn.template.BootstrapTemplate(
    title='Is my research used in clinical trials?'
)
template.main.append(
    pn.Column(
        input_doi,
        output_status,
        output_ratio
    )
)

template.servable();  # ; to prevent inline output / use preview instead